In [1]:
import time
import numpy as np
import pandas as pd

from Case import *
pd.set_option('display.max_columns',None)
# mimic = pd.read_csv("/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Make Derived Variable/mimic_df_cp.csv.gz", compression='gzip')
# eicu = pd.read_csv("/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Make Derived Variable/eicu_df_cp.csv.gz", compression='gzip')


# after_annoted_shock_m = annotation(mimic, 'mimic')
# mimic_labeled = optimized_shock_labeler(after_annoted_shock_m, 'mimic')

# after_annoted_shock_e = annotation(eicu, 'eicu')
# eicu_labeled = optimized_shock_labeler(after_annoted_shock_e, 'eicu')

# mimic_v1, eicu_v1 = filter_cohort(mimic, eicu) # 코호트의 모든 기록이 no_circ 인 경우 제외

# mimic_v2 = define_ambcirc(mimic_v1, 'mimic') # circ - amb - circ -> circ - amb_circ - circ
# eicu_v2 = define_ambcirc(eicu_v1, 'eicu')

# mimic_v3 = early_event_prediction_label(mimic_v2) # case 1, case 2 정의
# eicu_v3 = early_event_prediction_label(eicu_v2)

# mimic_v4 = optimized_recovered_labeler(mimic_v3, 'mimic') # case 3, case 4 정의
# eicu_v4 = optimized_recovered_labeler(eicu_v3, 'eicu')

# mimic_v5 = mimic_v4[~(mimic_v4['classes']=='undefined')]
# eicu_v5 = eicu_v4[~(eicu_v4['classes']=='undefined')]

In [31]:
# mimic_labeled.to_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip', index=False)
# eicu_labeled.to_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip', index=False)

In [2]:
mimic_labeled = pd.read_csv('check_point_mimic_labeled.csv.gz', compression = 'gzip')
eicu_labeled = pd.read_csv('check_point_eicu_labeled.csv.gz', compression = 'gzip')

In [5]:
import Case
from imp import reload
reload(Case)

<module 'Case' from 'c:\\Users\\DAHS\\Desktop\\ECP_CONT\\ECP_SCL\\Case Labeling\\Case.py'>

In [7]:
view_columns = ['Time_since_ICU_admission', 'MAP', 'Lactate', 'vasoactive/inotropic', 'progress', 'Annotation','Case']

In [6]:
mimic_case1_case2, mimic_case3, mimic_case4 = Case.Case_definetion(mimic_labeled, 'mimic')
eicu_case1_case2, eicu_case3, eicu_case4 = Case.Case_definetion(eicu_labeled, 'eicu')

|Start eICU-CRD process|
Extract Case 1, Case 2
--------------
Extract Case 3, Case 4


100%|██████████| 7624/7624 [01:06<00:00, 115.22it/s]


--------------
MAKE progress
--------------
Select target cohort
Finish, ....
--------------


In [19]:
any_circ_stay_ids = []
for stay_id, group in mimic_case1_case2.groupby('stay_id'):
    if any(group['Annotation'] == 'circ'):
        any_circ_stay_ids.append(stay_id)

m_case1_case2 = mimic_case1_case2[(mimic_case1_case2.stay_id.isin(any_circ_stay_ids))]

In [24]:
any_circ_stay_ids = []
for stay_id, group in eicu_case1_case2.groupby('patientunitstayid'):
    if any(group['Annotation'] == 'circ'):
        any_circ_stay_ids.append(stay_id)

e_case1_case2 = eicu_case1_case2[(eicu_case1_case2.patientunitstayid.isin(any_circ_stay_ids))]

In [32]:
m_case3 = mimic_case3.copy()
for stayid in m_case3['stay_id'].unique():
    targ_index = m_case3[m_case3['stay_id']==stayid].index
    
    if m_case3.loc[targ_index[-1]]['Case'] == 4:
        m_case3.drop(index=targ_index[-1], axis = 0, inplace = True)
    else:
        pass

In [54]:
all4_stay_ids = []
for stay_id, group in mimic_case4.groupby('stay_id'):
    if all(group['Case'] == 4):
        all4_stay_ids.append(stay_id)
        
m_case4 = mimic_case4[~(mimic_case4['stay_id'].isin(all4_stay_ids))]

In [33]:
e_case3 = eicu_case3.copy()
for stayid in e_case3['patientunitstayid'].unique():
    targ_index = e_case3[e_case3['patientunitstayid']==stayid].index
    
    if e_case3.loc[targ_index[-1]]['Case'] == 4:
        e_case3.drop(index=targ_index[-1], axis = 0, inplace = True)
    else:
        pass

In [56]:
all4_stay_ids = []
for stay_id, group in eicu_case4.groupby('patientunitstayid'):
    if all(group['Case'] == 4):
        all4_stay_ids.append(stay_id)
        
e_case4 = eicu_case4[~(eicu_case4['patientunitstayid'].isin(all4_stay_ids))]

In [35]:
invalid_columns_mimic, _ = find_invalid_columns(m_case1_case2)
invalid_columns_eicu, _ = find_invalid_columns(e_case1_case2)

m_case1_case2 = replace_inf_with_previous(m_case1_case2, invalid_columns_mimic)
e_case1_case2 = replace_inf_with_previous(e_case1_case2, invalid_columns_eicu)

In [36]:
invalid_columns_mimic, _ = find_invalid_columns(m_case3)
invalid_columns_eicu, _ = find_invalid_columns(e_case3)

m_case3 = replace_inf_with_previous(m_case3, invalid_columns_mimic)
e_case3 = replace_inf_with_previous(e_case3, invalid_columns_eicu)

In [62]:
nvalid_columns_mimic, _ = find_invalid_columns(m_case4)
invalid_columns_eicu, _ = find_invalid_columns(e_case4)

m_case4 = replace_inf_with_previous(m_case4, invalid_columns_mimic)
e_case4 = replace_inf_with_previous(e_case4, invalid_columns_eicu)

In [67]:
m_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
m_case3['INDEX'] = 'CASE3_DF'
m_case4['INDEX'] = 'CASE4_DF'

e_case1_case2['INDEX'] = 'CASE1_CASE2_DF'
e_case3['INDEX'] = 'CASE3_DF'
e_case4['INDEX'] = 'CASE4_DF'

In [76]:
case3_case4_m = pd.concat([m_case3.drop('Unnamed: 0', axis = 1), m_case4.drop('Unnamed: 0', axis = 1)], axis = 0)
mimic_analysis = pd.concat([m_case1_case2.drop('Unnamed: 0', axis = 1), case3_case4_m], axis = 0).reset_index(drop=True)

case3_case4_e = pd.concat([e_case3.drop('Unnamed: 0', axis = 1), e_case4.drop('Unnamed: 0', axis = 1)], axis = 0)
eicu_analysis = pd.concat([e_case1_case2.drop('Unnamed: 0', axis = 1), case3_case4_e], axis = 0).reset_index(drop=True)

In [77]:
def lactate_up(df, mode):
    targ = df.copy()
    
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    elif mode == 'eicu':
        stay_id_id = 'patientunitstayid'
        
    targ['lactate_up'] = np.nan
        
    for stayid in tqdm(targ[stay_id_id].unique()):
        
        stay_trajectory = targ[targ[stay_id_id]==stayid]

        true_measurements_lactate = stay_trajectory['Lactate'].unique()
        true_measurements_lactate_index = []
        
        for value in true_measurements_lactate:
            first_occurrence_index = stay_trajectory[stay_trajectory['Lactate'] == value].index[0]
            true_measurements_lactate_index.append(first_occurrence_index)
        
        #init
        
        anchor_lactate = true_measurements_lactate[0]
        count_change = 0
        
        # true_measurements_lactate_index = [0, 1, 2, 3]
        # true_measurements_lactate = [(1.2 masking), 1.2, 2.0, 1.4, 0.7]
        
        lactate_count_dict = {element: index+1 for index, element in enumerate(true_measurements_lactate)}
        
        for idx, row in stay_trajectory.iterrows():
            #LACTATE COND
            current_lactate = row['Lactate']
            
            if lactate_count_dict[current_lactate] - count_change == 1:
                lactate_decrease = current_lactate <= anchor_lactate
            
            elif current_lactate != anchor_lactate:
                count_change += 1
                lactate_decrease = current_lactate <= anchor_lactate #바뀌는 순간은 그대로 계산해도 됨
                for key, value in lactate_count_dict.items():
                    if value == count_change:
                        anchor_lactate = key
            
            # Annotation
            
            if lactate_decrease:
                targ.at[idx, 'lactate_up'] = 0
            else :
                targ.at[idx, 'lactate_up'] = 1

    return targ


mimic_analysis_save = lactate_up(mimic_analysis, 'mimic')
eicu_analysis_save = lactate_up(eicu_analysis, 'eicu')

100%|██████████| 10139/10139 [00:20<00:00, 491.73it/s]


In [163]:
# 차후 그래프 분석과 classification은 amb를 포함한 데이터인 아래 데이터를 그대로 사용
# 이어서 진행할 Contrastive learning은 amb 모두 제거

mimic_analysis_save.to_csv('mimic_analysis.csv.gz', compression = 'gzip', index=False)
eicu_analysis_save.to_csv('eicu_analysis.csv.gz', compression = 'gzip', index=False)

In [51]:
def print_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(dataset)}\n')
    print('샘플의 클래스 개수:')
    print(dataset['Case'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(dataset[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(dataset[patient_id_col].unique())}\n')

print_summary(mimic_analysis_save, 'MIMIC-IV', 'stay_id', 'subject_id')

print_summary(eicu_analysis_save, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 553634

샘플의 클래스 개수:
1.0    123954
2.0    129558
3.0     59283
4.0    240839
Name: Case, dtype: int64

총 Stay 수: 13626

총 환자 수: 12874

eICU CACE DEFINED

전체 샘플의 개수: 477479

샘플의 클래스 개수:
1.0    181863
2.0    131972
3.0     24653
4.0    138991
Name: Case, dtype: int64

총 Stay 수: 10148

총 환자 수: 9634



In [ ]:
# 이거 한번 해보고 찍어본 다음 샘플 수 확인 -> 너무 짜치면 아래 원하는 코호트만 선택하는거 진행

In [78]:
def train_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    df = dataset.copy()
    
    df_raw_case1_case2 = df[(df['INDEX']=='CASE1_CASE2_DF')&~(df['Annotation']=='circ')]
    df_raw_case3_case4 = df[~(df['INDEX']=='CASE1_CASE2_DF')]
    
    df = pd.concat([df_raw_case1_case2, df_raw_case3_case4], axis = 0).reset_index(drop=True)
    
    # df = df[~(df['Annotation']=='ambiguous')]
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(df)}\n')
    print('샘플의 클래스 개수:')
    print(df['Case'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(df[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(df[patient_id_col].unique())}\n')

train_summary(mimic_analysis_save, 'MIMIC-IV', 'stay_id', 'subject_id')
train_summary(eicu_analysis_save, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 476488

샘플의 클래스 개수:
1.0    123954
2.0    115949
3.0     59283
4.0    177302
Name: Case, dtype: int64

총 Stay 수: 13196

총 환자 수: 12496

eICU CACE DEFINED

전체 샘플의 개수: 386828

샘플의 클래스 개수:
1.0    181863
2.0    121846
3.0     24653
4.0     58466
Name: Case, dtype: int64

총 Stay 수: 9690

총 환자 수: 9233



In [73]:
mimic_analysis_save[mimic_analysis_save['INDEX']=='CASE3_DF']

4.0    240839
3.0     59283
Name: Case, dtype: int64

In [85]:
m_targ_stayid = set(m_case1_case2.stay_id.unique())&(set(m_case3.stay_id.unique()))
e_targ_stayid = set(e_case1_case2.patientunitstayid.unique())&(set(e_case3.patientunitstayid.unique()))

In [90]:
def train_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    df = dataset.copy()
    
    df_raw_case1_case2 = df[(df['INDEX']=='CASE1_CASE2_DF')&~(df['Annotation']=='circ')]
    df_raw_case3_case4 = df[(df['INDEX']=='CASE3_DF')]
    
    try:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.stay_id.isin(df_raw_case1_case2.stay_id.unique())]
    except:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.patientunitstayid.isin(df_raw_case1_case2.patientunitstayid.unique())]
    
    df = pd.concat([df_raw_case1_case2, df_raw_case3_case4], axis = 0).reset_index(drop=True)
    
    
    # df = df[~(df['Annotation']=='ambiguous')]
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(df)}\n')
    print('샘플의 클래스 개수:')
    print(df['Case'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(df[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(df[patient_id_col].unique())}\n')
    
    return df

mimic_analysis_save_gogo = train_summary(mimic_analysis_save, 'MIMIC-IV', 'stay_id', 'subject_id')
eicu_analysis_save_gogo = train_summary(eicu_analysis_save, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 280745

샘플의 클래스 개수:
1.0    123954
2.0    115949
3.0     19710
4.0     21132
Name: Case, dtype: int64

총 Stay 수: 12894

총 환자 수: 12231

eICU CACE DEFINED

전체 샘플의 개수: 320732

샘플의 클래스 개수:
1.0    181863
2.0    121846
3.0      8600
4.0      8423
Name: Case, dtype: int64

총 Stay 수: 9621

총 환자 수: 9171



In [93]:
mimic_analysis_save.to_csv('mimic_analysis_original.csv.gz', compression = 'gzip', index=False)
eicu_analysis_save.to_csv('eicu_analysis_original.csv.gz', compression = 'gzip', index=False)

In [92]:
mimic_analysis_save_gogo.to_csv('mimic_analysis.csv.gz', compression = 'gzip', index=False)
eicu_analysis_save_gogo.to_csv('eicu_analysis.csv.gz', compression = 'gzip', index=False)

In [97]:
view_columns = ['Time_since_ICU_admission', 'MAP', 'Lactate', 'lactate_up','vasoactive/inotropic', 'progress', 'Annotation','Case']

In [128]:
def train_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    df = dataset.copy()
    
    df_raw_case1_case2 = df[(df['INDEX']=='CASE1_CASE2_DF')&~(df['Annotation']=='circ')]
    df_raw_case3_case4 = df[(df['INDEX']=='CASE3_DF')]
    
    try:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.stay_id.isin(df_raw_case1_case2.stay_id.unique())]
    except:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.patientunitstayid.isin(df_raw_case1_case2.patientunitstayid.unique())]
    
    df = pd.concat([df_raw_case1_case2, df_raw_case3_case4], axis = 0).reset_index(drop=True)
    
    df = df[~((df['ABPd']==75.00)&(df['ABPs']==125.00))]
    
    
    # df = df[~(df['Annotation']=='ambiguous')]
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(df)}\n')
    print('샘플의 클래스 개수:')
    print(df['Case'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(df[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(df[patient_id_col].unique())}\n')
    
train_summary(mimic_analysis_save, 'MIMIC-IV', 'stay_id', 'subject_id')
train_summary(eicu_analysis_save, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 152089

샘플의 클래스 개수:
1.0    53845
2.0    58855
3.0    19688
4.0    19701
Name: Case, dtype: int64

총 Stay 수: 7789

총 환자 수: 7507

eICU CACE DEFINED

전체 샘플의 개수: 178331

샘플의 클래스 개수:
1.0    85028
2.0    76975
3.0     8596
4.0     7732
Name: Case, dtype: int64

총 Stay 수: 7307

총 환자 수: 7044



In [239]:
def train_summary(dataset, dataset_name, stay_id_col, patient_id_col):
    df = dataset.copy()
    
    df_raw_case1_case2 = df[(df['INDEX']=='CASE1_CASE2_DF')&~(df['Annotation']=='circ')]
    df_raw_case3_case4 = df[(df['INDEX']=='CASE3_DF')]
    
    try:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.stay_id.isin(df_raw_case1_case2.stay_id.unique())]
    except:
        df_raw_case3_case4 = df_raw_case3_case4[df_raw_case3_case4.patientunitstayid.isin(df_raw_case1_case2.patientunitstayid.unique())]
    
    df = pd.concat([df_raw_case1_case2, df_raw_case3_case4], axis = 0).reset_index(drop=True)
    
    df = df[~((df['ABPd']==75.00)&(df['ABPs']==125.00))]
    df = df[~(df['Annotation']=='ambiguous')]
    
    # df = df[~(df['Annotation']=='ambiguous')]
    print(f'{dataset_name} CACE DEFINED\n')
    print(f'전체 샘플의 개수: {len(df)}\n')
    print('샘플의 클래스 개수:')
    print(df['Case'].value_counts().sort_index())
    print(f'\n총 Stay 수: {len(df[stay_id_col].unique())}')
    print(f'\n총 환자 수: {len(df[patient_id_col].unique())}\n')
    
train_summary(mimic_analysis_save, 'MIMIC-IV', 'stay_id', 'subject_id')
train_summary(eicu_analysis_save, 'eICU', 'patientunitstayid', 'uniquepid')

MIMIC-IV CACE DEFINED

전체 샘플의 개수: 91513

샘플의 클래스 개수:
1.0    53845
2.0    19587
3.0     7022
4.0    11059
Name: Case, dtype: int64

총 Stay 수: 5055

총 환자 수: 4923

eICU CACE DEFINED

전체 샘플의 개수: 115879

샘플의 클래스 개수:
1.0    85028
2.0    24783
3.0     2340
4.0     3728
Name: Case, dtype: int64

총 Stay 수: 5636

총 환자 수: 5466



In [146]:
def lactate_up(df, mode):
    targ = df.copy().reset_index(drop=True)
    
    if mode == 'mimic':
        stay_id_id = 'stay_id'
    elif mode == 'eicu':
        stay_id_id = 'patientunitstayid'
        
    targ['lactate_up'] = np.nan
        
    for stayid in tqdm(targ[stay_id_id].unique()):
        
        stay_trajectory = targ[targ[stay_id_id]==stayid]

        true_measurements_lactate = stay_trajectory['Lactate'].unique()
        true_measurements_lactate_index = []
        
        for value in true_measurements_lactate:
            first_occurrence_index = stay_trajectory[stay_trajectory['Lactate'] == value].index[0]
            true_measurements_lactate_index.append(first_occurrence_index)
        
        #init
        
        anchor_lactate = true_measurements_lactate[0]
        count_change = 0
        
        # true_measurements_lactate_index = [0, 1, 2, 3]
        # true_measurements_lactate = [(1.2 masking), 1.2, 2.0, 1.4, 0.7]
        
        lactate_count_dict = {element: index+1 for index, element in enumerate(true_measurements_lactate)}
        
        for idx, row in stay_trajectory.iterrows():
            #LACTATE COND
            current_lactate = row['Lactate']
            
            if lactate_count_dict[current_lactate] - count_change == 1:
                lactate_decrease = current_lactate <= anchor_lactate
            
            elif current_lactate != anchor_lactate:
                count_change += 1
                lactate_decrease = current_lactate <= anchor_lactate #바뀌는 순간은 그대로 계산해도 됨
                for key, value in lactate_count_dict.items():
                    if value == count_change:
                        anchor_lactate = key
            
            # Annotation
            
            if lactate_decrease:
                targ.at[idx, 'lactate_up'] = 0
            else :
                targ.at[idx, 'lactate_up'] = 1

    return targ

In [148]:
mimic_analysis_save_please = lactate_up(mimic_analysis_save_gogo, 'mimic')
eicu_analysis_save_please = lactate_up(eicu_analysis_save_gogo, 'eicu')

100%|██████████| 7307/7307 [00:10<00:00, 720.56it/s]


In [184]:
case1_case2_reset = mimic_analysis_save_please[mimic_analysis_save_please['INDEX']=='CASE1_CASE2_DF'].reset_index(drop=True)
case3_case4_reset = mimic_analysis_save_please[~(mimic_analysis_save_please['INDEX']=='CASE1_CASE2_DF')].reset_index(drop=True)


case1_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='no_circ')&(case1_case2_reset['Shock_next_8h']==0)].index
amb_case1_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='ambiguous')&(case1_case2_reset['Shock_next_8h']==0)].index
case2_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='no_circ')&(case1_case2_reset['Shock_next_8h']==1)].index
amb_case2_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='ambiguous')&(case1_case2_reset['Shock_next_8h']==1)].index

case1_case2 = case1_case2_reset.copy()
case1_case2['Case'].loc[case1_idx] = 1
case1_case2['Case'].loc[amb_case1_idx] = 1
case1_case2['Case'].loc[case2_idx] = 2
case1_case2['Case'].loc[amb_case2_idx] = 2

mimic_please = pd.concat([case1_case2, case3_case4_reset], axis = 0).reset_index(drop=True)

In [187]:
case1_case2_reset = eicu_analysis_save_please[eicu_analysis_save_please['INDEX']=='CASE1_CASE2_DF'].reset_index(drop=True)
case3_case4_reset = eicu_analysis_save_please[~(eicu_analysis_save_please['INDEX']=='CASE1_CASE2_DF')].reset_index(drop=True)


case1_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='no_circ')&(case1_case2_reset['Shock_next_8h']==0)].index
amb_case1_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='ambiguous')&(case1_case2_reset['Shock_next_8h']==0)].index
case2_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='no_circ')&(case1_case2_reset['Shock_next_8h']==1)].index
amb_case2_idx = case1_case2_reset[(case1_case2_reset['Annotation']=='ambiguous')&(case1_case2_reset['Shock_next_8h']==1)].index

case1_case2 = case1_case2_reset.copy()
case1_case2['Case'].loc[case1_idx] = 1
case1_case2['Case'].loc[amb_case1_idx] = 1
case1_case2['Case'].loc[case2_idx] = 2
case1_case2['Case'].loc[amb_case2_idx] = 2

eicu_please = pd.concat([case1_case2, case3_case4_reset], axis = 0).reset_index(drop=True)

In [188]:
mimic_please.to_csv('mimic_analysis.csv.gz', compression = 'gzip', index=False)
eicu_please.to_csv('eicu_analysis.csv.gz', compression = 'gzip', index=False)

In [190]:
print(mimic_please.stay_id.nunique())
print(mimic_please.subject_id.nunique())

7789
7507


In [191]:
print(eicu_please.patientunitstayid.nunique())
print(eicu_please.uniquepid.nunique())

7307
7044


In [194]:
print(mimic_please.Case.value_counts().sort_index)

<bound method Series.sort_index of 1.0    76110
2.0    36590
4.0    19701
3.0    19688
Name: Case, dtype: int64>


In [195]:
print(eicu_please.Case.value_counts().sort_index)

<bound method Series.sort_index of 1.0    121515
2.0     40488
3.0      8596
4.0      7732
Name: Case, dtype: int64>


In [ ]:
#지금 하는 것도 잘 안되면 그냥 ambiguous 이면서 Case 2인 애들을 독립적인 label로 보자

In [207]:
mimic_please[~(mimic_please['Annotation']=='ambiguous')].Case.value_counts().sort_index

<bound method Series.sort_index of 1.0    53845
2.0    19587
4.0    11059
3.0     7022
Name: Case, dtype: int64>

In [208]:
eicu_please[~(eicu_please['Annotation']=='ambiguous')].Case.value_counts().sort_index

<bound method Series.sort_index of 1.0    85028
2.0    24783
4.0     3728
3.0     2340
Name: Case, dtype: int64>

In [236]:
target = mimic_please[~(mimic_please['Annotation']=='ambiguous')]
sample = target[target['stay_id']==target.stay_id.unique()[1436]]
sample[view_columns]

,Time_since_ICU_admission,MAP,Lactate,lactate_up,vasoactive/inotropic,progress,Annotation,Case,Lactate_fillna,Shock_next_8h
38496,0,74.333333,1.1,0.0,0,0,no_circ,2.0,0,1.0
38497,1,87.666667,1.1,0.0,0,0,no_circ,2.0,0,1.0
38498,2,75.000000,1.1,0.0,0,0,no_circ,2.0,0,1.0
38499,3,73.000000,1.1,0.0,0,0,no_circ,2.0,0,1.0
38500,4,76.333333,1.1,0.0,0,0,no_circ,2.0,0,1.0
38501,5,71.666667,1.1,0.0,0,0,no_circ,2.0,0,1.0
38502,6,71.333333,1.1,0.0,0,0,no_circ,2.0,0,1.0
